In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from CompLex import load_dataset, preprocess_data, tokenize_complex_dataset, create_base_model, apply_lora, create_trainer_complex
from train import train_model
from schema import TrainingConfig, TrainingTask

In [3]:

config = TrainingConfig(
    task=TrainingTask.CompLexV1,
    rank=16,
    alpha=16,
    target_modules=["query", "key", "value"],
    lora_dropout=0.1,
    learning_rate=2e-4,
    batch_size=8,
    num_epochs=3,
    max_input_length=128
)

In [4]:
model, tokenizer = create_base_model()
model = apply_lora(model, config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
data = load_dataset()
data = preprocess_data(data)
tokenized_dataset = tokenize_complex_dataset(data, tokenizer=tokenizer)

Filter:   0%|          | 0/9179 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9172 [00:00<?, ? examples/s]

Filter:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/9172 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

In [8]:
trainer = create_trainer_complex(
    model=model,
    config=config,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    output_dir="./outputs/first"
)

In [ ]:
train_time, vram = train_model(trainer)

In [ ]:
# Save model
MODEL_PATH = Path.home() / "models/bert" 

trainer.save_model(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)